In [0]:
jdbcHostname = "psql-mock-database-cloud.postgres.database.azure.com"
jdbcPort = "5432"
jdbcDatabase = "ecom1689955448469ndnuvtpfghatkriw"
username = "mtycxwjwdmzkvoljueonbbbi@psql-mock-database-cloud"
password = "tykvwdlkxmikoywfwgjebhmd"

jdbc_url = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"

connectionProperties = {
    "user": username,
    "password": password,
    "driver": "org.postgresql.Driver"
}

In [0]:
# Pergunta 1: Qual país possui a maior quantidade de itens cancelados?

sql_query = """
    SELECT c.country as país, SUM(od.quantity_ordered) AS soma_total_cancelados
    FROM customers c
    INNER JOIN orders o ON c.customer_number = o.customer_number
    INNER JOIN orderdetails od ON o.order_number = od.order_number
    WHERE o.status = 'Cancelled'
    GROUP BY país
    ORDER BY soma_total_cancelados DESC
    LIMIT 1
"""

df_q1 = spark.read.jdbc(url=jdbc_url, table=f"({sql_query}) AS subquery", properties=connectionProperties)

df_q1.show()

+-----+---------------------+
| país|soma_total_cancelados|
+-----+---------------------+
|Spain|                  605|
+-----+---------------------+



In [0]:
# Pergunta 2: Qual o faturamento da linha de produto mais vendido, considere os itens com status 'Shipped', cujo o pedido foi realizado no ano de 2005?

sql_query = """
SELECT pl.product_line, SUM(od.quantity_ordered * od.price_each) AS faturamento
FROM orders o
INNER JOIN orderdetails od ON o.order_number = od.order_number
INNER JOIN products p ON od.product_code = p.product_code
INNER JOIN product_lines pl ON p.product_line = pl.product_line
WHERE o.status = 'Shipped' AND o.order_date >= '2005-01-01' AND o.order_date < '2006-01-01'
GROUP BY pl.product_line
ORDER BY faturamento DESC
LIMIT 1
"""

df_q2 = spark.read.jdbc(url=jdbc_url, table=f"({sql_query}) AS subquery", properties=connectionProperties)

df_q2.show()

+------------+--------------------+
|product_line|         faturamento|
+------------+--------------------+
|Classic Cars|603666.9900000000...|
+------------+--------------------+



In [0]:
# Pergunta 3: Nome, sobrenome e email dos vendedores do Japão, o local-part do e-mail deve estar mascarado.

sql_query = """
SELECT
    e.first_name as Nome,
    e.last_name as sobrenome,
    SPLIT_PART(e.email, '@', 1) || '@*****' AS email
FROM
    offices o
INNER JOIN
    employees e ON o.office_code = e.office_code
WHERE
    o.country = 'Japan'
"""

df_q3 = spark.read.jdbc(url=jdbc_url, table=f"({sql_query}) AS subquery", properties=connectionProperties)

df_q3.show()   

+-------+---------+------------+
|   nome|sobrenome|       email|
+-------+---------+------------+
|   Mami|    Nishi|mnishi@*****|
|Yoshimi|     Kato| ykato@*****|
+-------+---------+------------+



In [0]:
# Salve os resultados em formato delta


df_q1.write.format("delta").mode("overwrite").saveAsTable("question1")
df_q2.write.format("delta").mode("overwrite").saveAsTable("question2")
df_q3.write.format("delta").mode("overwrite").saveAsTable("question3")